### Notebook contains 

The prompts that are taken from the sample usecases inside vertex ai for study

In [1]:
!pip install google-cloud-aiplatform transformers sentence-transformers faiss-cpu langchain > /dev/null

In [ ]:
!pip freeze | grep aiplatform

google-cloud-aiplatform==1.25.0


In [1]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [2]:
import vertexai
from vertexai.preview.language_models import TextGenerationModel

In [3]:
def predict_large_language_model_sample(
    project_id: str,
    model_name: str,
    temperature: float,
    max_decode_steps: int,
    top_p: float,
    top_k: int,
    content: str,
    location: str = "us-central1",
    tuned_model_name: str = "",
    ) :
    """Predict using a Large Language Model."""
    vertexai.init(project=project_id, location=location)
    model = TextGenerationModel.from_pretrained(model_name)
    if tuned_model_name:
      model = model.get_tuned_model(tuned_model_name)

    response = model.predict(
        content,
        temperature=temperature,
        max_output_tokens=max_decode_steps,
        top_k=top_k,
        top_p=top_p,)

    print(f"Response from Model: {response.text}")

    return response.text

In [ ]:
# Support rep chat summarization

prompt_birth = '''Name of the character Michael karthik who is affectionately called as mike by his schoolmates.
Create a 300 word back story revolving around his child hood. 
Speaks about his family, his life in suburban city in a developed country.'''

In [ ]:
reply_birth = predict_large_language_model_sample("generativeaitrial", 
                                    "text-bison@001", 
                                    0.2, 
                                    256, 
                                    0.8, 
                                    40,
                                    prompt_birth)

In [ ]:
with open('michaelKarthik.txt',mode='w',encoding='utf-8') as mic:
  mic.write(reply_birth)

In [ ]:
# Create Linkedin Articles

prompt_posts ='''Write 10 linkedin posts that would be written Michael.  
Below is the information about michael
Michael spend hours playing video games and learning about how computers. 
He realized eventually that computer was his passion.
After high school, Michael went to college to study computer science. 
He graduated with honors and got a job as a software engineer'''

In [ ]:
reply_posts = predict_large_language_model_sample("generativeaitrial", 
                                    "text-bison@001", 
                                    0.2, 
                                    1024, 
                                    0.8, 
                                    40,
                                    prompt_posts)

In [ ]:
with open('michaelKarthik.txt',
          mode='a+',
          encoding='utf-8') as mic:
  mic.write(reply_posts)

In [ ]:
prompt_medium ='''Write 10 medium articles that would be written Michael.  
Below is the information about michael
Michael spend hours working on Machine learning and algorithms. 
He realized python and assembly language his major study area.
Michael is a computer science graduate working as a software engineer'''

In [ ]:
reply_articles = predict_large_language_model_sample("generativeaitrial", 
                                    "text-bison@001", 
                                    0.2, 
                                    1024, 
                                    0.8, 
                                    40,
                                    prompt_medium)

In [ ]:
with open('michaelKarthik.txt',
          mode='a+',
          encoding='utf-8') as mic:
  mic.write('\n')
  mic.write(reply_articles)

In [ ]:
prompt_tweets = """Michael likes to hang out with friends and 
engages with his friends and followers on Twitter. Michael is smart 
savvy about algorithms and likes to joke about AI on twitter. 
Based on above information write 25 tweets written by Michael"""

In [ ]:
reply_tweets = predict_large_language_model_sample("generativeaitrial", 
                                    "text-bison@001", 
                                    0.2, 
                                    1024, 
                                    0.8, 
                                    40,
                                    prompt_tweets)

In [ ]:
with open('michaelKarthik.txt',
          mode='a+',
          encoding='utf-8') as mic:
  mic.write('\n')
  mic.write(reply_tweets)

In [4]:
prompt_prefs = """Michael lived in city and enjoys hanging out with 
friends, dating females and chilling out at beach. His preferences 
are sophisticated and mature. Write 200 words about his prefernces on 
cars, colors, food, favorite places, hobbies, art, music, movies 
and clothing"""

In [ ]:
reply_prefs = predict_large_language_model_sample("generativeaitrial", 
                                    "text-bison@001", 
                                    0.2, 
                                    1024, 
                                    0.8, 
                                    40,
                                    prompt_prefs)

With the data ready next we can start the vector stores embedding

In [6]:
with open('michaelKarthik.txt',
          mode='a+',
          encoding='utf-8') as mic:
  mic.write('\n')
  mic.write(reply_prefs)

In [7]:
with open('/content/michaelKarthik.txt') as f:
  mike = f.read()

In [9]:
from langchain.text_splitter import NLTKTextSplitter
nltk_splitter = NLTKTextSplitter(chunk_size=500)

In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
nltk_docs = nltk_splitter.split_text(mike)

print(len(nltk_docs))
nltk_docs[0:3]

41


["Michael Karthik was born in a suburban city in a developed country to Indian parents.\n\nHis father was a software engineer and his mother was a stay-at-home mom.\n\nMichael had one older sister and one younger brother.\n\nMichael's childhood was happy and carefree.\n\nHe loved playing with his siblings and friends.\n\nHe was also a good student and enjoyed learning new things.\n\nMichael's parents were always supportive of him and encouraged him to follow his dreams.",
 "He loved playing with his siblings and friends.\n\nHe was also a good student and enjoyed learning new things.\n\nMichael's parents were always supportive of him and encouraged him to follow his dreams.\n\nWhen Michael was in high school, he started to get interested in computers.\n\nHe would spend hours playing video games and learning about how computers worked.\n\nMichael's parents were initially worried about his obsession with computers, but they eventually realized that it was his passion.",
 "Michael's parent

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings 

embed = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [13]:
from langchain.vectorstores import FAISS

db = FAISS.from_texts(nltk_docs, embed)

In [14]:
query = "How was Michael childhood"
docs = db.similarity_search(query)

In [23]:
def build_convPrompt(question):
  """Gets the supporting documents from Mikes data"""
  docs = db.similarity_search(question)
  text = ''
  for loc in docs:
    text = text + loc.page_content

  prompt = f""" {question} 
  Reply to the question by thinking step by step. 
  You can use the following context that is collected from Michael's past data. 
  {text}
  You can share your thoughts before completing the reply
  End the reply with 10 word question related to discussion"""

  return prompt

In [24]:
query1 = "Today was interesting, and enjoyable. Hows was your day Michael?"

query1_prompt = build_convPrompt(query1)

print(query1_prompt)

 Today was interesting, and enjoyable. Hows was your day Michael? 
  Reply to the question by thinking step by step. 
  You can use the following context that is collected from Michael's past data. 
  Michael quickly rose through the ranks and became a successful engineer.

Michael is now married with two children.

He lives in a nice house in the suburbs and drives a luxury car.

Michael is grateful for his successful career and happy family life.

He knows that he is lucky to have such a great life.

Michael's story is an inspiration to others.

It1.

**Michael's first LinkedIn post**

I'm excited to announce that I've accepted a new position as a software engineer at Google!Thank you to everyone who has supported me along the way.

I couldn't have done it without you!

2.

**Michael's post about his first day at work**

My first day at Google was an amazing experience!

I was welcomed by a warm and friendly team, and I was immediately given the opportunity to contribute to a project

In [25]:
query1_reply = predict_large_language_model_sample("generativeaitrial", 
                                    "text-bison@001", 
                                    0.2, 
                                    1024, 
                                    0.8, 
                                    40,
                                    query1_prompt)

Response from Model: My day was good, thanks for asking. I had a productive day at work and I got to spend some time with my family in the evening. How was your day?


In [19]:
query1_reply

"My day was great! I got to work on a new project that I'm really excited about. How was your day?"

In [26]:
query2 = "My day was bit hectic and had a rough time with my boss"

query2_prompt = build_convPrompt(query2)

print(query2_prompt)

 My day was bit hectic and had a rough time with my boss 
  Reply to the question by thinking step by step. 
  You can use the following context that is collected from Michael's past data. 
  Thank you to everyone who has supported me along the way.

I couldn't have done it without you!

2.

**Michael's post about his first day at work**

My first day at Google was an amazing experience!

I was welcomed by a warm and friendly team, and I was immediately given the opportunity to contribute to a project that I'm passionate about.

I'm learning so much every day, and I'm excited to see what the future holds.9.

**Michael's post about a promotion he received**

I'm excited to announce that I've been promoted to Senior Software Engineer!

I'm grateful for the opportunity to continue growing my career at Google, and I'm looking forward to taking on new challenges.

I'm also grateful for the support of my colleagues and managers.

I couldn't have achieved this without them.

10.

**Michael's 

In [27]:
query2_reply = predict_large_language_model_sample("generativeaitrial", 
                                    "text-bison@001", 
                                    0.2, 
                                    1024, 
                                    0.8, 
                                    40,
                                    query2_prompt)

Response from Model: Sorry to hear that. Hope your day gets better. Do you want to talk about it?


In [22]:
print(query2_reply)

What do you think about the new project you're working on?
